<a href="https://colab.research.google.com/github/znurtokatli/final-project/blob/zehra/ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-31 23:46:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.34MB/s    in 0.2s    

2021-05-31 23:46:24 (5.34 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
data_url = "https://uncc-datavizbc-finalproject-data.s3.us-east-2.amazonaws.com/Data/"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

from pyspark import SparkFiles

In [5]:
# Load in heart_clean.csv 
url = data_url + "heart_clean.csv"
spark.sparkContext.addFile(url)

heart_df = spark.read.option('header', 'true').csv(SparkFiles.get("heart_clean.csv"), inferSchema=True, sep=',', timestampFormat="m/dd/yy")
heart_df.show(10)

+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
|age|sex| cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
+---+---+---+------+----+---+-------+--------+----+-------+---+---+-----+------+
| 63|  1|  3|   145| 233|  1|      0|     150|   0|    2.3|  0|  0|    1|     1|
| 37|  1|  2|   130| 250|  0|      1|     187|   0|    3.5|  0|  0|    2|     1|
| 41|  0|  1|   130| 204|  0|      0|     172|   0|    1.4|  2|  0|    2|     1|
| 56|  1|  1|   120| 236|  0|      1|     178|   0|    0.8|  2|  0|    2|     1|
| 57|  0|  0|   120| 354|  0|      1|     163|   1|    0.6|  2|  0|    2|     1|
| 57|  1|  0|   140| 192|  0|      1|     148|   0|    0.4|  1|  0|    1|     1|
| 56|  0|  1|   140| 294|  0|      0|     153|   0|    1.3|  1|  0|    2|     1|
| 44|  1|  1|   120| 263|  0|      1|     173|   0|    0.0|  2|  0|    3|     1|
| 52|  1|  2|   172| 199|  1|      1|     162|   0|    0.5|  2|  0|    3|     1|
| 57|  1|  2|   150| 168|  0

In [6]:
# Load in stroke_clean.csv 
url = data_url + "stroke_clean.csv" 
spark.sparkContext.addFile(url)

stroke_df = spark.read.option('header', 'true').csv(SparkFiles.get("stroke_clean.csv"), inferSchema=True, sep=',', timestampFormat="m/dd/yy")
stroke_df.show(10)

+-----+------+----+------------+-------------+------------+---------+--------------+-----------------+----+--------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level| bmi|smoking_status|stroke|
+-----+------+----+------------+-------------+------------+---------+--------------+-----------------+----+--------------+------+
| 9046|     1|67.0|           0|            1|           1|        1|             1|           228.69|36.6|             0|     1|
|51676|     0|61.0|           0|            0|           1|        2|             0|           202.21|null|             1|     1|
|31112|     1|80.0|           0|            1|           1|        1|             0|           105.92|32.5|             1|     1|
|60182|     0|49.0|           0|            0|           1|        1|             1|           171.23|34.4|             2|     1|
| 1665|     0|79.0|           1|            0|           1|        2|             0|      

In [7]:
# Load in insurance_clean.csv 
url = data_url + "insurance_clean.csv" 
spark.sparkContext.addFile(url)

stroke_df = spark.read.option('header', 'true').csv(SparkFiles.get("insurance_clean.csv"), inferSchema=True, sep=',', timestampFormat="m/dd/yy")
stroke_df.show(10)

+---+---+------+--------+------+------------------+
|age|sex|   bmi|children|smoker|           charges|
+---+---+------+--------+------+------------------+
| 19|  0|  27.9|       0|     1|         16884.924|
| 18|  1| 33.77|       1|     0|         1725.5523|
| 28|  1|  33.0|       3|     0|          4449.462|
| 33|  1|22.705|       0|     0|       21984.47061|
| 32|  1| 28.88|       0|     0|         3866.8552|
| 31|  0| 25.74|       0|     0|         3756.6216|
| 46|  0| 33.44|       1|     0|         8240.5896|
| 37|  0| 27.74|       3|     0|         7281.5056|
| 37|  1| 29.83|       2|     0|         6406.4107|
| 60|  0| 25.84|       0|     0|28923.136919999997|
+---+---+------+--------+------+------------------+
only showing top 10 rows



In [ ]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade
# install joblib & kernas. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib
#!pip install keras.utils
#!pip install keras.models

In [ ]:
import pandas as pd
